In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [4]:
pretrain_features = pd.read_csv('./pretrain_features.csv', index_col=0)
pretrain_labels = pd.read_csv('./pretrain_labels.csv', index_col=0)

train_features = pd.read_csv('./train_features.csv', index_col=0)
train_labels = pd.read_csv('./train_labels.csv', index_col=0)

test_features = pd.read_csv('./test_features.csv', index_col=0)

In [5]:
X_pretrain = pretrain_features.iloc[:,1:]
y_pretrain = pretrain_labels
X_train = train_features.iloc[:80,1:]
y_train = train_labels.iloc[:80,:]
X_test = test_features.iloc[:,1:]

In [7]:
from sklearn.preprocessing import RobustScaler
scale = RobustScaler()
X_pretrain = scale.fit_transform(X_pretrain)
X_train = scale.transform(X_train)
X_test = scale.transform(X_test)


In [8]:
# Make a model with X layers
layer1 = keras.layers.Dense(128, activation="relu")
layer2 = keras.layers.Dense(256, activation="relu")
layer3 = keras.layers.Dense(512, activation="relu")
layer4 = keras.layers.Dropout(0.2)
layer5 = keras.layers.Dense(1024, activation="relu")
layer6 = keras.layers.Dense(512, activation="relu")
layer7 = keras.layers.Dense(256, activation="relu")
layer8 = keras.layers.Dense(128, activation="relu")
layer9 = keras.layers.Dense(128, activation="linear")
base_model = keras.Sequential([keras.Input(shape=(1000,)), layer1, layer2, layer3, layer4, layer5, layer6, layer7, layer8, layer9])

# Train the model
base_model.compile(optimizer="adam", loss="mse")
base_model.fit(X_pretrain, y_pretrain, epochs=20, batch_size=32)

Epoch 1/20
1563/1563 [==============================] - 18s 11ms/step - loss: 0.0930
Epoch 2/20
1563/1563 [==============================] - 19s 12ms/step - loss: 0.0107
Epoch 3/20
1563/1563 [==============================] - 15s 10ms/step - loss: 0.0076
Epoch 4/20
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0066
Epoch 5/20
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0055
Epoch 6/20
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0047
Epoch 7/20
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0042
Epoch 8/20
1563/1563 [==============================] - 17s 11ms/step - loss: 0.0033
Epoch 9/20
1563/1563 [==============================] - 17s 11ms/step - loss: 0.0031
Epoch 10/20
1563/1563 [==============================] - 17s 11ms/step - loss: 0.0028
Epoch 11/20
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0025
Epoch 12/20
1563/1563 [==============================] - 17s 11

In [9]:
base_model.trainable = False
inputs = keras.Input(shape=(1000,))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning
x = base_model(inputs, training=False)
x = keras.layers.Dense(128, activation="relu")(x)
x = keras.layers.Dense(256, activation="relu")(x)
x = keras.layers.Dense(512, activation="relu")(x)
x = keras.layers.Dense(1024, activation="relu")(x)
x = keras.layers.Dense(512, activation="relu")(x)
x = keras.layers.Dense(256, activation="relu")(x)
x = keras.layers.Dense(128, activation="relu")(x)
x = keras.layers.Dense(32, activation="relu")(x)
outputs = keras.layers.Dense(1, activation="linear")(x)
model = keras.Model(inputs, outputs)

In [10]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.mse,
              metrics=[keras.metrics.mean_squared_error])
model.fit(X_train, y_train, epochs=500, batch_size=16, verbose=0)

In [11]:
from sklearn.metrics import mean_squared_error
y_preds = model.predict(X_train)
print("mse:", mean_squared_error(y_train, y_preds, squared=False))

mse: 0.26171818619095133


In [12]:
X_val = train_features.iloc[80:,1:]
y_val = train_labels.iloc[80:,:]
from sklearn.metrics import mean_squared_error
y_preds_val = model.predict(X_val)
print("mse:", mean_squared_error(y_val, y_preds_val, squared=False))

mse: 0.2551331005042021


In [13]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are take into account
model.compile(optimizer=keras.optimizers.Adam(1e-4),  # Very low learning rate
              loss=keras.losses.mse,
              metrics=[keras.metrics.mean_squared_error])

# Train end-to-end. Be careful to stop before you overfit!
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
3/3 [==============================] - 1s 14ms/step - loss: 0.0648 - mean_squared_error: 0.0648
Epoch 2/500
3/3 [==============================] - 0s 17ms/step - loss: 0.0689 - mean_squared_error: 0.0689
Epoch 3/500
3/3 [==============================] - 0s 15ms/step - loss: 0.0612 - mean_squared_error: 0.0612
Epoch 4/500
3/3 [==============================] - 0s 14ms/step - loss: 0.0652 - mean_squared_error: 0.0652
Epoch 5/500
3/3 [==============================] - 0s 13ms/step - loss: 0.0599 - mean_squared_error: 0.0599
Epoch 6/500
3/3 [==============================] - 0s 14ms/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 7/500
3/3 [==============================] - 0s 13ms/step - loss: 0.0620 - mean_squared_error: 0.0620
Epoch 8/500
3/3 [==============================] - 0s 13ms/step - loss: 0.0592 - mean_squared_error: 0.0592
Epoch 9/500
3/3 [==============================] - 0s 13ms/step - loss: 0.0578 - mean_squared_error: 0.0578
Epoch 10/500
3/3 [==========

In [14]:
from sklearn.metrics import mean_squared_error
y_preds = model.predict(X_train)
print("mse:", mean_squared_error(y_train, y_preds, squared=False))

mse: 0.001261137135427026


In [15]:
X_val = train_features.iloc[80:,1:]
y_val = train_labels.iloc[80:,:]
from sklearn.metrics import mean_squared_error
y_preds_val = model.predict(X_val)
print("mse:", mean_squared_error(y_val, y_preds_val, squared=False))

mse: 0.23319183111078828


In [135]:
submission = model.predict(X_test)
test_features["y"] = submission
to_submit = test_features[[ "y"]]
to_submit.to_csv("./submission.csv")